In [ ]:
# NO RUNNEAR MÁS ESTE NOTEBOOK
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/Shareddrives/Deep Learning/DeepLearning_2022/Final Project/Data/'

df = pd.read_csv(data_path + 'songdata.csv')

Mounted at /content/drive


In [ ]:
data = ', '.join(df['text'])
data[:369]
chars = sorted(list(set(data)))
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [ ]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

In [ ]:
#define the length of the input and output sequence:
seq_length = 25  

#set the seed value:
seed_value = 42
tf.random.set_seed(seed_value)
random.seed(seed_value)

hidden_layer = 256
learning_rate = 0.001 

In [ ]:
tf.compat.v1.disable_eager_execution()

inputs = tf.compat.v1.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.compat.v1.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

init_state = tf.compat.v1.placeholder(shape=[1, hidden_layer], dtype=tf.float32, name="state")

In [ ]:
# input to hidden layer weights
U_i = tf.Variable(tf.compat.v1.truncated_normal([vocab_size, hidden_layer], stddev=0.05))
# hidden to hidden layer weights
W_i = tf.Variable(tf.compat.v1.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
# bias for hidden layer
b_i = tf.Variable(tf.zeros([hidden_layer]))

U_f = tf.Variable(tf.compat.v1.truncated_normal([vocab_size, hidden_layer], stddev=0.05))
W_f = tf.Variable(tf.compat.v1.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
b_f = tf.Variable(tf.zeros([hidden_layer]))

U_o = tf.Variable(tf.compat.v1.truncated_normal([vocab_size, hidden_layer], stddev=0.05))
W_o = tf.Variable(tf.compat.v1.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
b_o = tf.Variable(tf.zeros([hidden_layer]))

U_g = tf.Variable(tf.compat.v1.truncated_normal([vocab_size, hidden_layer], stddev=0.05))
W_g = tf.Variable(tf.compat.v1.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
b_g = tf.Variable(tf.zeros([hidden_layer]))

# hidden layer to output weights
V = tf.Variable(tf.compat.v1.truncated_normal([hidden_layer, vocab_size], stddev=0.05))
# bias to output
b_v = tf.Variable(tf.zeros([1]))

In [ ]:
def LSTM_cell(input, prev_hidden_state, prev_cell_state): # LSTM model

    it = tf.sigmoid(tf.matmul(input, U_i) + tf.matmul(prev_hidden_state, W_i) + b_i)

    ft = tf.sigmoid(tf.matmul(input, U_f) + tf.matmul(prev_hidden_state, W_f) + b_f)

    ot = tf.sigmoid(tf.matmul(input, U_o) + tf.matmul(prev_hidden_state, W_o) + b_o)

    gt = tf.tanh(tf.matmul(input, U_g) + tf.matmul(prev_hidden_state, W_g) + b_g)

    ct = (prev_cell_state * ft) + (it * gt)

    ht = ot * tf.tanh(ct)

    return ct, ht

In [ ]:
with tf.compat.v1.variable_scope("LSTM") as scope:
    #initialize hidden state and cell state for each batch
    hidden_state = init_state
    cell_state = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()  

        cell_state, hidden_state = LSTM_cell(x_t, hidden_state, cell_state)
    
        y_hat.append(tf.matmul(hidden_state, V) + b_v)

In [ ]:
output_softmax = tf.nn.softmax(y_hat[-1])  

outputs = tf.concat(y_hat, axis=0)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

hprev = hidden_state

gradients = tf.gradients(loss, tf.compat.v1.trainable_variables())
clipped, _ = tf.clip_by_global_norm(gradients, 4.0)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).apply_gradients(zip(clipped, tf.compat.v1.trainable_variables()))

sess = tf.compat.v1.Session()
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

pointer = 0
iteration = 0

In [ ]:
# NO RUNNEAR MÁS
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_layer])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, optimizer],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1

Se han truncado las últimas 5000 líneas del flujo de salida.
He shine  
That jall offo lasta  
Hindie's I can have a mistake is country  
If I love you on  
Here we are jelicied up the plans  
The ways are not supfison.  
Shadows are been in importing  
And a bo 

-------------------------------------------------------------------------------------------------------------------


 After 323000 iterations

 ains, baby, is face and I need a canary  
Oh let my cracks that closed  
'No little time to see him  
Until I didn't hopely make it for  
I feeling time in the ends  
To think I would see knockin'  
And there goes to our bell  
I've got tears in your stake  
In this ried  
Don't are a like to spepa  
And time step to live in the pawe  
You must all ala-baby don't have somethin'  
Down with no rotterasire  
You think not you sweal again  
I guess who's together and a little bottom  
Alled as in o 

---------------------------------------------------------------------------------------